In [55]:
## ADVENT OF CODE 2022, Day 21
## Edmund Dickinson, Python implementation

# File read
input_folder = "Input/"
input_file = "day21a.txt"
file_path = input_folder + input_file

with open(file_path) as file:
    input = file.read().splitlines()

In [56]:
class Monkey:
    def __init__(self, tribe, id, oper_str):
        self.tribe = tribe
        self.id = id
        self.val = None        
        self.operation = None
        self.operands = [None, None]
        self.parseOperation(oper_str)

    def parseOperation(self, oper_str):
        if oper_str.isnumeric():
            # Set a value
            self.val = int(oper_str)
        else:
            # Set an expression
            self.operands[0] = oper_str[:4]
            self.operands[1] = oper_str[7:]
            self.operation   = oper_str[5]

    def doOperation(self):
        a,b = tuple([self.tribe[i].val for i in self.operands])
        
        if   self.operation == '+':
            self.val = a + b
        elif self.operation == '-':
            self.val = a - b
        elif self.operation == '*':
            self.val = a * b
        elif self.operation == '/':
            # Assumed floor division for ints
            self.val = a // b

    def doReverseOperation(self, known):
        if (known == 'a'):
            source = self.operands[0]
            dest   = self.operands[1]
        if (known == 'b'):
            source   = self.operands[1]
            dest     = self.operands[0]
        
        y = self.tribe[source].val
        
        if   self.operation == '=':
            self.tribe[dest].val = y
        elif self.operation == '+':
            self.tribe[dest].val = self.val - y
        elif self.operation == '*':
            self.tribe[dest].val = self.val // y        
        elif self.operation == '-':
            if (known == 'a'):
                self.tribe[dest].val = y - self.val
            if (known == 'b'):
                self.tribe[dest].val = y + self.val
        elif self.operation == '/':
            # Assumed floor division for ints
            if (known == 'a'):
                self.tribe[dest].val = y // self.val
            if (known == 'b'):
                self.tribe[dest].val = y * self.val            

    def checkComputation(self):
        if (self.id == "humn"):
            return

        if (self.operation == "="):
            self.checkReverseComputation()
        elif self.val is None:
            # Forward computation
            if not (self.tribe[self.operands[0]].val is None
                    or
                    self.tribe[self.operands[1]].val is None
            ):
                self.doOperation()
        else:
            if not (self.operation is None):
                self.checkReverseComputation()

    def checkReverseComputation(self):
        # Is one of the operands known? If so, compute the value by solving inverse problem.
        if (not (self.tribe[self.operands[0]].val is None)):
            self.doReverseOperation('a')
        elif (not (self.tribe[self.operands[1]].val is None)):
            self.doReverseOperation('b')            

In [57]:
# File parsing
def parseMonkeys(inputs):
    monkeys = {}

    for line in input:
        id = line.split(": ")[0]
        oper_str = line.split(": ")[1]

        monkeys[id] = Monkey(monkeys, id, oper_str)

    return monkeys

In [58]:
# First puzzle
monkeys = parseMonkeys(input)

# Pass through all monkeys repeatedly until all values are filled in
while monkeys["root"].val is None:
    for monkey in monkeys.values():
        monkey.checkComputation()

print ( monkeys["root"].val )

194501589693264


In [59]:
# Second puzzle
monkeys = parseMonkeys(input)

# Apply corrections to parsing assumptions:

# root monkey is a "match" condition
monkeys["root"].operation = "="

# "humn" is the value to be determined
monkeys["humn"].val = None
monkeys["humn"].operands = [None, None]

while monkeys["humn"].val is None:
    for monkey in monkeys.values():
        monkey.checkComputation()

print ( monkeys["humn"].val )

3887609741189
